In [1]:
import zipfile

def unzip_file(file_path):
  """Giải nén tệp zip.

  Args:
    file_path: Đường dẫn đến tệp zip.
  """
  with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall()

# Giải nén tệp zip đầu tiên
unzip_file('/content/VLSP2021.zip')

# Giải nén tệp zip thứ hai
unzip_file('/content/Testing.zip')

In [2]:
import os
import re

def process_text(text):
    """Xử lý nội dung văn bản để phân tách và gán các cột với nhãn B- và I- cho cả cột chính và nhãn lồng."""
    tokens = re.split(r'(\s+|<ENAMEX.*?>|</ENAMEX>)', text)
    tag_stack = []
    result = []
    inside_entity = False  # Cờ cho nhãn chính (cột 2)
    inside_nested_entity = False  # Cờ cho nhãn lồng (cột 3)

    for token in tokens:
        token = token.strip()
        if not token:
            continue

        if token.startswith('<ENAMEX TYPE='):
            # Bắt đầu một thực thể mới
            current_tag = re.search(r'TYPE="([^"]+)"', token).group(1)
            tag_stack.append(current_tag)
            if len(tag_stack) == 1:
                inside_entity = False  # Nhãn chính
            elif len(tag_stack) == 2:
                inside_nested_entity = False  # Nhãn lồng
        elif token == '</ENAMEX>':
            # Kết thúc một thực thể
            if tag_stack:
                tag_stack.pop()
            if len(tag_stack) == 1:
                inside_nested_entity = False
            if len(tag_stack) == 0:
                inside_entity = False
        else:
            # Cột 2: Nhãn chính (B- và I-)
            if tag_stack:
                main_tag = tag_stack[0]
                ner_tag = f"B-{main_tag}" if not inside_entity else f"I-{main_tag}"
                inside_entity = True
            else:
                ner_tag = "O"

            # Cột 3: Nhãn lồng (B- và I- của nhãn thứ 2)
            if len(tag_stack) > 1:
                nested_tag = tag_stack[1]
                nested_ner_tag = f"B-{nested_tag}" if not inside_nested_entity else f"I-{nested_tag}"
                inside_nested_entity = True
            else:
                nested_ner_tag = "0"

            # Thêm kết quả (token, nhãn chính, nhãn lồng)
            result.append([token, ner_tag, nested_ner_tag])

    return result

def process_files_to_single_file(input_dir, output_file):
    """Xử lý tất cả các tệp .muc trong thư mục đầu vào và dồn vào một tệp duy nhất."""
    with open(output_file, 'w', encoding='utf-8') as output:
        for filename in os.listdir(input_dir):
            if filename.endswith('.muc'):
                input_path = os.path.join(input_dir, filename)

                # Đọc nội dung tệp
                with open(input_path, 'r', encoding='utf-8') as file:
                    text = file.read()

                # Xử lý văn bản
                processed_data = process_text(text)

                for row in processed_data:
                    output.write(" ".join(row) + "\n")
                output.write("\n")  # Dòng trống giữa các tệp

# Đường dẫn thư mục và tệp đầu ra
input_directory = "/content/Train-Muc"  # Thay bằng đường dẫn thư mục
output_file = "/content/train.txt"  # Thay bằng đường dẫn tệp kết quả

# Gọi hàm xử lý
process_files_to_single_file(input_directory, output_file)


In [3]:
# Đường dẫn thư mục và tệp đầu ra
input_directory = "/content/Test"  # Thay bằng đường dẫ
output_file = "/content/test.txt"  # Thay bằng đường dẫn tệp kết quả

# Gọi hàm xử lý
process_files_to_single_file(input_directory, output_file)


In [4]:
!pip install python-crfsuite


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.0 MB/s eta 0:00:00


In [21]:
import pycrfsuite
import nltk
from nltk.tokenize import word_tokenize

# Đọc dữ liệu từ train.txt và chuyển thành định dạng phù hợp
def read_data(file_path):
    sentences = []
    sentence = []

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            # Loại bỏ các dòng trống
            if line.strip():
                word, tag, tag_nested = line.strip().split()
                # Loại bỏ tiền tố B- hoặc I- nếu có trong tag
                clean_tag = tag # Giữ phần sau dấu '-'
                sentence.append((word, clean_tag))
            else:
                if sentence:
                    sentences.append(sentence)
                    sentence = []

    # Đảm bảo đưa câu cuối vào
    if sentence:
        sentences.append(sentence)

    return sentences


# Chuyển các từ và nhãn thành đặc trưng cho CRF
def word2features(sentence, i):
    word = sentence[i][0]
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word.isupper()': word.isupper(),
        'word.isdigit()': word.isdigit(),
        'word.istitle()': word.istitle(),
        'word[-3:]': word[-3:],
        'word[:3]': word[:3],
    }
    if i > 0:
        word1 = sentence[i - 1][0]
        features.update({
            'prev.word.lower()': word1.lower(),
            'prev.word.isupper()': word1.isupper(),
        })
    else:
        features['BOS'] = True  # Beginning of sentence

    if i < len(sentence) - 1:
        word1 = sentence[i + 1][0]
        features.update({
            'next.word.lower()': word1.lower(),
            'next.word.isupper()': word1.isupper(),
        })
    else:
        features['EOS'] = True  # End of sentence

    return features

# Chuyển dữ liệu thành đặc trưng và nhãn
def prepare_data(sentences):
    X = []
    y = []
    for sentence in sentences:
        X.append([word2features(sentence, i) for i in range(len(sentence))])
        y.append([tag for _, tag in sentence])
    return X, y

# Đọc dữ liệu từ file
train_sentences = read_data('/content/train.txt')

# Chuẩn bị dữ liệu đặc trưng
X_train, y_train = prepare_data(train_sentences)

# Huấn luyện mô hình CRF
trainer = pycrfsuite.Trainer()
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Tinh chỉnh các tham số huấn luyện (nếu cần thiết)
trainer.set_params({
    'max_iterations': 50,
    'feature.possible_transitions': True,

})

# Lưu mô hình đã huấn luyện
trainer.train('ner_model.crfsuite')

# Dự đoán với mô hình đã huấn luyện
def predict(model_file, sentence):
    tagger = pycrfsuite.Tagger()
    tagger.open(model_file)

    # Chuyển câu thành đặc trưng
    X_test = [word2features(sentence, i) for i in range(len(sentence))]
    return tagger.tag(X_test)

# Ví dụ dự đoán
test_sentence = ["Moscow", "là", "thủ", "đô", "của", "Nga"]
predicted_tags = predict('ner_model.crfsuite', test_sentence)

# In kết quả dự đoán
for word, tag in zip(test_sentence, predicted_tags):
    print(f'{word}: {tag}')


Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 155540
Seconds required: 1.960

L-BFGS optimization
c1: 0.000000
c2: 1.000000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 800752.970297
Feature norm: 5.000000
Error norm: 170032.680378
Active features: 155540
Line search trials: 2
Line search step: 0.000007
Seconds required for this iteration: 46.926

***** Iteration #2 *****
Loss: 654528.869844
Feature norm: 4.155477
Error norm: 142070.910076
Active features: 155540
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 15.676

***** Iteration #3 *****
Loss: 608083.678544
Feature norm: 3.858745
Error norm: 95912.462003
Active features: 155540
Line search trials: 2
Line search step: 0.074128
Seconds re

In [22]:
X_train[0]

[{'bias': 1.0,
  'word.lower()': 'sau',
  'word.isupper()': False,
  'word.isdigit()': False,
  'word.istitle()': True,
  'word[-3:]': 'Sau',
  'word[:3]': 'Sau',
  'BOS': True,
  'next.word.lower()': "'sống",
  'next.word.isupper()': False},
 {'bias': 1.0,
  'word.lower()': "'sống",
  'word.isupper()': False,
  'word.isdigit()': False,
  'word.istitle()': True,
  'word[-3:]': 'ống',
  'word[:3]': "'Số",
  'prev.word.lower()': 'sau',
  'prev.word.isupper()': False,
  'next.word.lower()': 'chung',
  'next.word.isupper()': False},
 {'bias': 1.0,
  'word.lower()': 'chung',
  'word.isupper()': False,
  'word.isdigit()': False,
  'word.istitle()': False,
  'word[-3:]': 'ung',
  'word[:3]': 'chu',
  'prev.word.lower()': "'sống",
  'prev.word.isupper()': False,
  'next.word.lower()': 'với',
  'next.word.isupper()': False},
 {'bias': 1.0,
  'word.lower()': 'với',
  'word.isupper()': False,
  'word.isdigit()': False,
  'word.istitle()': False,
  'word[-3:]': 'với',
  'word[:3]': 'với',
  'prev.

In [23]:
len(X_train)

1050

In [24]:
y_train[0]

['O',
 'B-PRODUCT',
 'I-PRODUCT',
 'I-PRODUCT',
 'I-PRODUCT',
 'I-PRODUCT',
 'O',
 'O',
 'O',
 'B-PRODUCT',
 'I-PRODUCT',
 'I-PRODUCT',
 'I-PRODUCT',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-PRODUCT',
 'I-PRODUCT',
 'I-PRODUCT',
 'I-PRODUCT',
 'I-PRODUCT',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-PRODUCT',
 'I-PRODUCT',
 'I-PRODUCT',
 'I-PRODUCT',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-PERSONTYPE',
 'I-PERSONTYPE',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-PRODUCT',
 'I-PRODUCT',
 'I-PRODUCT',
 'I-PRODUCT',
 'I-PRODUCT',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-PRODUCT',
 'I-PRODUCT',
 'I-PRODUCT',
 'I-PRODUCT',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-PERSONTYPE',
 'I-PERSONTYPE',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LOCATION-GPE',
 'I-LOCATION-GPE',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',


In [25]:
sentence = [("Moscow", "LOC"), ("là", "O"), ("thủ", "O"), ("đô", "O"), ("của", "O"), ("Nga", "LOC")]
i = 0  # Index of the word "Moscow"

In [26]:
print(X_train[0] )

[{'bias': 1.0, 'word.lower()': 'sau', 'word.isupper()': False, 'word.isdigit()': False, 'word.istitle()': True, 'word[-3:]': 'Sau', 'word[:3]': 'Sau', 'BOS': True, 'next.word.lower()': "'sống", 'next.word.isupper()': False}, {'bias': 1.0, 'word.lower()': "'sống", 'word.isupper()': False, 'word.isdigit()': False, 'word.istitle()': True, 'word[-3:]': 'ống', 'word[:3]': "'Số", 'prev.word.lower()': 'sau', 'prev.word.isupper()': False, 'next.word.lower()': 'chung', 'next.word.isupper()': False}, {'bias': 1.0, 'word.lower()': 'chung', 'word.isupper()': False, 'word.isdigit()': False, 'word.istitle()': False, 'word[-3:]': 'ung', 'word[:3]': 'chu', 'prev.word.lower()': "'sống", 'prev.word.isupper()': False, 'next.word.lower()': 'với', 'next.word.isupper()': False}, {'bias': 1.0, 'word.lower()': 'với', 'word.isupper()': False, 'word.isdigit()': False, 'word.istitle()': False, 'word[-3:]': 'với', 'word[:3]': 'với', 'prev.word.lower()': 'chung', 'prev.word.isupper()': False, 'next.word.lower()': 

In [27]:
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support

# Hàm đọc dữ liệu test từ file
def read_test_data(file_path):
    sentences = []
    sentence = []

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            # Loại bỏ các dòng trống
            if line.strip():
                word, tag = line.strip().split()[:2]
                sentence.append((word, tag))
            else:
                if sentence:
                    sentences.append(sentence)
                    sentence = []

    # Đảm bảo đưa câu cuối vào
    if sentence:
        sentences.append(sentence)

    return sentences

# Chuẩn bị dữ liệu test
test_sentences = read_test_data('/content/test.txt')

# Dự đoán trên tập test
def predict_test_data(model_file, test_sentences):
    tagger = pycrfsuite.Tagger()
    tagger.open(model_file)

    X_test = [[word2features(sentence, i) for i in range(len(sentence))] for sentence in test_sentences]
    y_test = [[tag for _, tag in sentence] for sentence in test_sentences]
    y_pred = [tagger.tag(xseq) for xseq in X_test]

    return y_test, y_pred

# Dự đoán trên tập dữ liệu test
y_test, y_pred = predict_test_data('ner_model.crfsuite', test_sentences)

# Chuyển dữ liệu thành dạng phẳng để tính các chỉ số
y_test_flat = [tag for sentence in y_test for tag in sentence]
y_pred_flat = [tag for sentence in y_pred for tag in sentence]

In [28]:
# # Danh sách mới để lưu kết quả
# y_pred_flat_bio = []

# # Biến tạm lưu nhãn trước đó
# previous_label = None

# # Duyệt qua từng nhãn trong danh sách
# for i, label in enumerate(y_pred_flat):
#     if label == 'O':
#         y_pred_flat_bio.append('O')  # Giữ nguyên nếu là nhãn 'O'
#         previous_label = None  # Đặt lại nhãn trước
#     else:
#         # Nếu nhãn hiện tại khác nhãn trước đó hoặc sau 'O', gán B-
#         if label != previous_label:
#             y_pred_flat_bio.append(f"B-{label}")
#         else:
#             # Nếu nhãn giống nhãn trước đó, gán I-
#             y_pred_flat_bio.append(f"I-{label}")
#         previous_label = label  # Cập nhật nhãn trước đó


In [29]:
# y_pred_flat = y_pred_flat_bio

In [30]:
# Sắp xếp lại nhãn để các nhãn B- và I- của cùng một loại xếp cạnh nhau
unique_tags = sorted(set(y_test_flat + y_pred_flat), key=lambda x: (x[2:], x[:2]))  # Sắp xếp theo loại nhãn và B/I

# Báo cáo kết quả
print("Classification Report:")
print(classification_report(y_test_flat, y_pred_flat, labels=unique_tags, digits=4))

# Tính Precision, Recall, và F1
precision, recall, f1, _ = precision_recall_fscore_support(y_test_flat, y_pred_flat, average='weighted')
print(f"Weighted Precision: {precision:.4f}")
print(f"Weighted Recall: {recall:.4f}")
print(f"Weighted F1-Score: {f1:.4f}")

Classification Report:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_

                       precision    recall  f1-score   support

                    O     0.9274    0.9875    0.9565    172690
            B-ADDRESS     0.0000    0.0000    0.0000        23
            I-ADDRESS     0.0000    0.0000    0.0000       248
           B-DATETIME     0.4725    0.4528    0.4624       625
           I-DATETIME     0.3986    0.5068    0.4462       663
      B-DATETIME-DATE     0.8821    0.3735    0.5248       581
      I-DATETIME-DATE     0.8902    0.4261    0.5763       514
 B-DATETIME-DATERANGE     0.6000    0.0845    0.1481       142
 I-DATETIME-DATERANGE     0.7021    0.1553    0.2543       425
  B-DATETIME-DURATION     0.8203    0.4939    0.6166       490
  I-DATETIME-DURATION     0.8502    0.4694    0.6049       556
       B-DATETIME-SET     0.0000    0.0000    0.0000         4
       I-DATETIME-SET     0.0000    0.0000    0.0000        20
      B-DATETIME-TIME     0.6667    0.0370    0.0702        54
      I-DATETIME-TIME     0.6667    0.0282    0.0541  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [32]:
import pycrfsuite

# Hàm để trích xuất các đặc trưng từ một câu
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

# Hàm để chuẩn bị dữ liệu đầu vào cho mô hình
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

# Hàm để dự đoán nhãn cho một câu
def predict_tags(model_path, sentence):
    tagger = pycrfsuite.Tagger()
    tagger.open(model_path)

    # Tách câu thành các từ
    words = sentence.split()

    # Tạo dữ liệu đầu vào cho mô hình
    features = sent2features([(word, 'X') for word in words]) # 'X' là một postag giả định

    # Dự đoán nhãn
    tags = tagger.tag(features)

    return tags

# Đường dẫn đến mô hình đã được đào tạo
model_path = '/content/ner_model.crfsuite'

# Câu muốn thử nghiệm
test_sentence = "tên tôi là Đinh Mạnh Cường"

# Dự đoán nhãn cho câu
predicted_tags = predict_tags(model_path, test_sentence)

# In kết quả
for word, tag in zip(test_sentence.split(), predicted_tags):
    print(f'{word}: {tag}')

tên: O
tôi: O
là: O
Đinh: B-PERSON
Mạnh: I-PERSON
Cường: I-PERSON
